In [1]:
# importing the required modules and libraries
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
# loading the dataset
iris = datasets.load_iris()

In [3]:
# assigning data to X and target to Y
X = iris.data
Y = iris.target
features = np.array([1,2,3,4])

In [4]:
# splitting the data into training and testing
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 1)

In [5]:
def print_leafNode(y,level):
    if(len(list(set(y)))==1):
        count_value= list(set(y))[0]
        count= np.count_nonzero(y==count_value)
        print("Level ",level)
        print("Count of ",count_value," ",count)
        print("Current Entropy  is = 0.0")
        print("Reached leaf Node")
        print()
    else:
        length= len(list(set(y)))
        print("Level ",level)
        for i in range(length):
            
            count_value= list(set(y))[i]
            count= np.count_nonzero(y==count_value)
            print("Count of ",count_value," ",count)
        print("Current Entropy  is = 0.0")
        print("Reached leaf Node")
        print()

In [6]:
def print_node(count0, count1, count2, totalCount, index, gainRatio, level, features):
    print("Level ",level)
    print("Count of 0 = ",count0)
    print("Count of 1 = ",count1)
    print("Count of 2 = ",count2)
    print("Current Entropy  is = ",informationN(count0,count1,count2))
    print("Splitting on feature X",features[index],"with gain ratio",gainRatio)
    print()

In [7]:
def countOfValuesInParentNode(Y):    # finding the total values belonging to differnet classes in the parent node
    count0, count1, count2, totalCount = 0,0,0,0
    for i in range(len(Y)):
        if(Y[i] == 0):
            count0 += 1
        elif(Y[i] == 1):
            count1 += 1
        elif(Y[i] == 2):
            count2 += 1
    totalCount  = count0 + count1 + count2
    return count0, count1, count2, totalCount

In [8]:
# finding the information of Nth Node   
def informationN(count0, count1, count2):
    totalCount = count0 + count1 + count2
    a = count0/totalCount
    b = count1/totalCount
    c = count2/totalCount
    if(a == 0):
        a = 1
    if(b == 0):
        b = 1
    if(c == 0):
        c = 1   
    value = -(((a)*(np.log10(a))) + ((b)*(np.log10(b))) + ((c)*(np.log10(c))))
    return value

In [9]:
def splitInfo(countLeftNode,countRightNode,totalValuesInX):    # function to calculate split info
    a = countLeftNode/totalValuesInX
    b = countRightNode/totalValuesInX

    value = -(((a) * (np.log10(a))) + ((b) * (np.log10(b))))
    return value

In [10]:
def leftNodeAndRightNodeValues(mean,X,Y):
    countLeft0, countLeft1, countLeft2, countRight0, countRight1, countRight2 = 0, 0, 0, 0, 0, 0
    for i in range(len(X)):
        if(X[i] < mean and Y[i] == 0):
            countLeft0 += 1
        elif(X[i] < mean and Y[i] == 1):
            countLeft1 += 1
        elif(X[i] < mean and Y[i] == 2):
            countLeft2 += 1
        elif(X[i] >= mean and Y[i] == 0):
            countRight0 += 1
        elif(X[i] >= mean and Y[i] == 1):
            countRight1 += 1
        elif(X[i] > mean and Y[i] == 2):
            countRight2 += 1

    return countLeft0, countLeft1, countLeft2, countRight0, countRight1, countRight2

In [11]:
def function3(mean,X,Y):      # finding the gain ratio
    # for information in Nth Node
    countLeft0, countLeft1, countLeft2, countRight0, countRight1, countRight2 = leftNodeAndRightNodeValues(mean,X,Y)
    totalCount = countLeft0 + countLeft1 + countLeft2 + countRight0 + countRight1 + countRight2
    
    # information for nth node
    information_N = informationN(countLeft0 + countRight0, countLeft1 + countRight1, countLeft2 + countRight2)
    
    # information for left child node
    totalCountInLeft = countLeft0 + countLeft1 + countLeft2
    information_left_child_node = informationN(countLeft0,countLeft1,countLeft2)
    
    # information for right child node
    totalCountInRight = countRight0 + countRight1 + countRight2
    information_right_child_node = informationN(countRight0, countRight1, countRight2)
    
    # information in both the child nodes combined
    information_child_nodes = ((totalCountInLeft/totalCount)*(information_left_child_node)) + ((totalCountInRight/totalCount)*(information_right_child_node))
    
    # information gain from the parent node and the child nodes
    information_gain = information_N - information_child_nodes
    
    # finding the split info by calling the split info function
    split_info = 0
    split_info = splitInfo(totalCountInLeft,totalCountInRight,totalCount)
    
    # calculating the gain ratio
    gain_ratio = (information_gain/split_info)
    
    return gain_ratio

In [12]:
def function2(X,Y):    # for finding the mean taht gives the best gain ratio in a particular feature
    distinct_values_in_X = list(set(X))  # finding unique values in X[:,i]
    value = -100000
    mean_to_be_returned = -1
    length_of_set_X = len(distinct_values_in_X)
    for i in range(length_of_set_X-1):    # finding the gain ratio for mean of two consecutive values
        mean = (distinct_values_in_X[i] + distinct_values_in_X[i+1])/2
        output = function3(mean,X,Y)
        if(output > value):     # checking the best gain ratio
            value = output
            mean_to_be_returned = mean
    return mean_to_be_returned
#     return mean

In [13]:
def function1(X,Y):     # to check which is the best feature
    value = -100000    
    feature = -1
    mean_sent_for_original_splitting = -1
    for i in range(len(X[0])):   # passing every feature one by one
        mean_returned = function2(X[:,i],Y)    # using the means which are returned to decide the best feature by finding gain ratio for every mean returned 
        
        output = function3(mean_returned,X[:,i],Y)     # gain ratio for means of different features
        if(output > value):        # finding the best feature through mean
            value = output
            mean_sent_for_original_splitting = mean_returned
            feature = X[:,i]
            index = i
    return mean_sent_for_original_splitting,feature,index     # returning the best mean of the best feature for original splitting

In [14]:
def run(X_train,Y_train,features,level):  

    # features getting exhausted
    if(len(X_train[0]) == 0):
#     if(len(features) == 0):
        print_leafNode(Y_train,level)
        return
    
    # pure node
    elif(len(list(set(Y_train))) == 1):
        print_leafNode(Y_train,level)
        return

    mean, feature, index = function1(X_train,Y_train)
    
    gain_ratio = function3(mean,feature,Y_train)
    
    count0, count1, count2, totalCount = countOfValuesInParentNode(Y_train)
    
    print_node(count0, count1, count2, totalCount, index, gain_ratio, level,features)
    
    left_child_array = []
    right_child_array = []
    X_left_child = []
    X_right_child = []
    Y_left_child = []
    Y_right_child = []
    for i in range(len(X_train)):
        if(feature[i] < mean):
            left_child_array.append(i)
        elif(feature[i] > mean):
            right_child_array.append(i)

             
    for i in left_child_array:
        X_left_child.append(X_train[i])
        Y_left_child.append(Y_train[i])
    
    for i in right_child_array:
        X_right_child.append(X_train[i])
        Y_right_child.append(Y_train[i])
        
    X_left_child = np.delete(X_left_child,index,axis = 1)
    X_right_child = np.delete(X_right_child,index,axis = 1)
    features = np.delete(features,index)
    

    run(X_left_child,Y_left_child,features,level+1)
    run(X_right_child,Y_right_child,features,level+1)

In [15]:
run(X_train,Y_train,features,0)

Level  0
Count of 0 =  37
Count of 1 =  34
Count of 2 =  41
Current Entropy  is =  0.4758440486038902
Splitting on feature X 3 with gain ratio 1.0

Level  1
Count of  0   37
Current Entropy  is = 0.0
Reached leaf Node

Level  1
Count of 0 =  0
Count of 1 =  34
Count of 2 =  41
Current Entropy  is =  0.29913564599242237
Splitting on feature X 4 with gain ratio 0.620732397093067

Level  2
Count of 0 =  0
Count of 1 =  33
Count of 2 =  5
Current Entropy  is =  0.16910438509969486
Splitting on feature X 2 with gain ratio 0.051397950830260455

Level  3
Count of 0 =  0
Count of 1 =  28
Count of 2 =  5
Current Entropy  is =  0.18471773080933496
Splitting on feature X 1 with gain ratio 0.09658358100919552

Level  4
Count of  1   27
Count of  2   4
Current Entropy  is = 0.0
Reached leaf Node

Level  4
Count of  1   1
Count of  2   1
Current Entropy  is = 0.0
Reached leaf Node

Level  3
Count of  1   5
Current Entropy  is = 0.0
Reached leaf Node

Level  2
Count of 0 =  0
Count of 1 =  1
Count of